In [8]:
import os, cv2, keras
import pandas as pd
import numpy as np
import tensorflow as tf
import math
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
train_dir = "sign_data_ver1/train"
test_dir = "sign_data_ver1/test"
train_csv = "sign_data_ver1/train_data.csv"
test_csv =  "sign_data_ver1/test_data.csv"

In [3]:
# Dataset
# Note: Implement a shuffle system

class Dataset(tf.keras.utils.Sequence):
    def __init__(self, csv = None, dir = None, batch_size = None, resize = None):
        self.df = pd.read_csv(csv)
        self.df.columns = ["img1", "img2", "label"]

        self.img1 = self.df["img1"]
        self.img2 = self.df["img2"]
        self.label = self.df["label"]

        self.dir = dir
        self.resize = resize
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)

    def __getitem__(self, index):
        batch_x1 = self.img1[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x2 = self.img2[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.label[index * self.batch_size:(index + 1) * self.batch_size]

        X1, X2 = [], []
        for x1, x2 in zip(batch_x1, batch_x2):
            img1_path = os.path.join(self.dir, x1)
            img2_path = os.path.join(self.dir, x2)

            img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

            if self.resize is not None:
                img1 = cv2.resize(img1, (self.resize))
                img2 = cv2.resize(img2, (self.resize))

            X1.append(img1)
            X2.append(img2)

        return [np.array(X1), np.array(X2)], np.array(batch_y)

In [3]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

In [5]:
# from keras.layers import *
# from keras.models import Model, Sequential 

# img1_inp = Input((train_data.resize),name="img1_inp")
# img2_inp = Input((train_data.resize),name="img2_inp")

# network = Sequential()

# network.add(Conv2D(64,(3,3),input_shape=(224, 224, 1),activation='relu'))
# network.add(MaxPooling2D(3, strides=2))

# network.add(Conv2D(32,(3,3),activation='relu'))
# network.add(MaxPooling2D(2,2))

# network.add(Flatten())
# network.add(Dense(128,activation = 'relu'))

# feature_vector_1 = network(img1_inp)
# feature_vector_2 = network(img2_inp)

# concat = Concatenate()([feature_vector_1, feature_vector_2])
# dense = Dense(128, activation='relu')(concat)
# output = Dense(1, activation='sigmoid')(dense)

# network = Model(inputs=[img1_inp, img2_inp], outputs=output)
# network.summary()    

In [9]:
from keras import layers
from keras import Input
from keras import regularizers

input = layers.Input((128, 128, 1))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(32, (5, 5), activation="relu",kernel_regularizer=regularizers.L2(l2=2e-4),
    bias_regularizer=regularizers.L2(2e-4))(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(32, (5, 5),kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(2e-4), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)


x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(128, activation="relu")(x)
embedding_network = tf.keras.Model(input, x)


input_1 = layers.Input((128, 128, 1))
input_2 = layers.Input((128, 128, 1))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [10]:
from keras.optimizers import Adam

# network.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=['accuracy'])
siamese.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
siamese.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 128)          3578980     ['input_3[0][0]',                
                                                                  'input_4[0][0]']          

In [8]:
from keras.callbacks import EarlyStopping 

# network.fit(train_data, epochs=20, validation_data=test_data, callbacks=EarlyStopping(patience=3))

train_data = Dataset(train_csv, train_dir, batch_size=32, resize=[128, 128])
test_data = Dataset(test_csv, test_dir, batch_size=32, resize=[128, 128])

siamese.fit(
    train_data,
    validation_data = test_data,
    epochs = 10,
    verbose = 1,
)

Epoch 1/10
726/726 [==============================] - 927s 1s/step - loss: 0.7319 - accuracy: 0.5287 - val_loss: 0.4942 - val_accuracy: 0.8845
Epoch 2/10
726/726 [==============================] - 903s 1s/step - loss: 0.6953 - accuracy: 0.5465 - val_loss: 0.5712 - val_accuracy: 0.8457
Epoch 3/10
726/726 [==============================] - 915s 1s/step - loss: 0.6910 - accuracy: 0.5513 - val_loss: 0.6086 - val_accuracy: 0.7797
Epoch 4/10
726/726 [==============================] - 911s 1s/step - loss: 0.6879 - accuracy: 0.5573 - val_loss: 0.5463 - val_accuracy: 0.8864
Epoch 5/10
726/726 [==============================] - 933s 1s/step - loss: 0.6861 - accuracy: 0.5588 - val_loss: 0.5311 - val_accuracy: 0.8949
Epoch 6/10
726/726 [==============================] - 914s 1s/step - loss: 0.6854 - accuracy: 0.5619 - val_loss: 0.5027 - val_accuracy: 0.7125
Epoch 7/10
726/726 [==============================] - 920s 1s/step - loss: 0.6846 - accuracy: 0.5614 - val_loss: 0.5339 - val_accuracy: 0.8869

In [5]:
siamese.save("siamese_model1.hdf5")
siamese.save_weights("siamese_model1_weight.h5")

NameError: name 'siamese' is not defined

In [10]:
siamese.predict(test_data)

180/180 [==============================] - 68s 375ms/step


array([[0.8708989 ],
       [0.9385685 ],
       [0.89968365],
       ...,
       [0.49877053],
       [0.46944293],
       [0.5098684 ]], dtype=float32)

In [24]:
siamese.evaluate(test_data, verbose=1)

180/180 [==============================] - 65s 359ms/step - loss: 0.4403 - accuracy: 0.8192


AttributeError: 'list' object has no attribute 'flatten'

In [62]:
test1 = "sign_data_ver1/test/049/01_049.png"
img1 = cv2.imread(test1, cv2.IMREAD_GRAYSCALE)
img1 = cv2.resize(img1, (128 , 128))

test2 = "sign_data_ver1/test/050/12_050.png"
img2 = cv2.imread(test2, cv2.IMREAD_GRAYSCALE)
img2 = cv2.resize(img2, (128 , 128))

siamese.predict([img1.reshape((1, 128, 128)), 
                img2.reshape((1, 128, 128))])

1/1 [==============================] - 0s 51ms/step


array([[0.5460352]], dtype=float32)

In [20]:
from keras.models import load_model

# model = load_model('siamese_model1.h5')
model = load_model('siamese_model1.h5', custom_objects={'euclidean_distance': euclidean_distance})

SystemError: Exception encountered when calling layer "lambda" (type Lambda).

unknown opcode

Call arguments received by layer "lambda" (type Lambda):
  • inputs=['tf.Tensor(shape=(None, 128), dtype=float32)', 'tf.Tensor(shape=(None, 128), dtype=float32)']
  • mask=None
  • training=False